In [1]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.6 MB 10.7 MB/s 
     |████████████████████████████████| 636 kB 70.3 MB/s 
     |████████████████████████████████| 895 kB 60.5 MB/s 
     |████████████████████████████████| 3.3 MB 31.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 13.9 MB/s 



 '사업명', '사업_부처명', '계속과제여부', '내역사업명', '과제명', '요약문_연구목표','요약문_연구내용', '요약문_기대효과', '요약문_한글키워드'

       번역 완료 : 사업명 1-50,000
       번역 해야할 것 : 과제명 1-10,000

In [2]:
import pandas as pd
import numpy as np
import re
import time
import os

# from tqdm import tqdm
from tqdm.notebook import tqdm

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [12]:
input_dir = "gdrive/MyDrive/dataset/open/"
train_data = "train.csv"
## 수정 (요약문_연구내용, 요약문_연구목표 두개가 가장 김) '요약문_연구목표', '요약문_연구내용',
column_name = ['요약문_연구내용']

start_index = 70001
end_index = 80000

# batch_size가 길고, column을 한번에 많이 불러오면 뒷부분이 번역이 짤림 (주의!)
# 추천 batch_size = 6  & column = 1 (컬럼이 길지 않으면 2개씩 하거나, batch_size를 높여도 좋을듯)
batch_size = 6

df = pd.read_csv(input_dir+train_data)
df = pd.DataFrame(df)
df = df.fillna("내용없음")

df = df[start_index:end_index+1]
df = df[column_name].values.tolist()
df = sum(df, [])

In [13]:
# set gpu device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [14]:
model = M2M100ForConditionalGeneration.from_pretrained(
    "facebook/m2m100_418M").to(device)

tokenizer = M2M100Tokenizer.from_pretrained(
    "facebook/m2m100_418M",
    src_lang="ko", 
    tgt_lang="en")

In [15]:
def chunk_batch(iterable, batch_size):
  length = len(iterable)
  result= []
  for ndx in range(0, length,  batch_size):
    result.append(iterable[ndx : min(ndx + batch_size, length)])
  return result

def list_chunk(list_, n):
  return [list_[i:i+n] for i in range(0, len(list_), n)]

In [16]:
def m2m100(data):
  sample = []
  tokenizer.src_lang = "ko"
  # print(1)
  # print(data) 
  encoded_ko = tokenizer(data, return_tensors="pt", padding=True).to(device)
  # start = time.time()    
  generated_tokens = model.generate(
        **encoded_ko, 
        forced_bos_token_id=tokenizer.get_lang_id("en"), 
        num_beams=5, 
        no_repeat_ngram_size=4, 
        repetition_penalty=1.2, 
        length_penalty=0.7).to(device)        
  # print(f"{len(data)} timecheck : {time.time() - start}") 

  
  return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [17]:
split_data = chunk_batch(df, batch_size)

In [ ]:
result = []
save_term_length = len(split_data)//10

for idx, data in enumerate(tqdm(split_data, total=len(split_data))):
  result.append(m2m100(data))

  if idx % save_term_length == 0:
    target_list = list_chunk(sum(result, []), len(column_name))
    target_df = pd.DataFrame(target_list, columns=column_name)
    target_df.to_csv("gdrive/MyDrive/dataset/open/target_batch_{}_{}_{}.csv".format(column_name, start_index, end_index))
  
target_list = list_chunk(sum(result, []), len(column_name))
target_df = pd.DataFrame(target_list, columns=column_name)
target_df.to_csv("gdrive/MyDrive/dataset/open/target_batch_{}_{}_{}.csv".format(column_name, start_index, end_index))

Token indices sequence length is longer than the specified maximum sequence length for this model (1054 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
target_df

,요약문_연구내용
0,In this study we want to develop a total of fo...
1,1st year: Expression of LPA and ESM-1 Expressi...
2,"For the purposes of this Regulation, the Europ..."
3,"For this purpose, we will develop a practical ..."
4,No content
...,...
19996,○ (1th year)Tourism Content DB Construction an...
19997,"1) The first year of the study, the first year..."
19998,(1) Pitch 80um React Members Spring Detection ...
19999,1.Community Garden or Type Design Development ...


In [ ]:
target_df[['요약문_기대효과']]

요약문_기대효과    • Improving the rapid, accurate sympathy of ne...
Name: 0, dtype: object